In [1]:
import mailbox
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import re
import os
import string
import random
import time
import requests
import pprint
import urllib.request
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [2]:
def split_sentence(text):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    content = '\n'.join(tokenizer.tokenize(text))
    return content

https://sitereport.netcraft.com/?url=https://www.paypal.com/us

# is_url_malicious

In [3]:
url_dict  = {}
def is_url_malicious(url):
    # netcraft - site used for internet data mining
    base_url = 'http://toolbar.netcraft.com/site_report?url=' + url
    driver = webdriver.Chrome(executable_path=r"chromedriver.exe")
    driver.get(base_url)
    # obtain details from site - html parsed
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    try:
        # find span tags in extracted html code with classname risk label
        risk = soup.find("span", class_="risk_label")
        risk = risk.get('data-risk')
        if(int(risk) > 1):
            url_dict[url] = True
            return True
        url_dict[url] = False
        return False
    except:
        return None

# single mail mailicious

In [4]:
#this is finding the urls. you can change the name of the function
def single_mail_malicious(message):
    """
    input: a single mail
    output: true if the mail contains malicious link, false other wise, None when the mail has a wrong format

    Given give a single mail, perform link analysis, and check whether it contains malicious link or not
    """
    body = None
    links_list = list()
    if message.is_multipart():
        for part in message.walk():
            if part.is_multipart():
                for subpart in part.walk():
                    if 'text/html' in subpart.get_content_type():
                        body = subpart
                    elif subpart.get_content_type() == 'text/plain':
                        body = subpart
                    else:
                        return None
            elif 'text/html' in part.get_content_type():
                body = part
            elif part.get_content_type() == 'text/plain':
                body = part
            else:
                return None

    elif 'text/html' in message.get_content_type():
        body = message
    elif message.get_content_type() == 'text/plain':
        body = message
    else:
        return None

    if (body == None or body.get_payload() == None):
        return None
    elif 'text/html' in body.get_content_type() or body.get_content_type() == "text/plain":
        new_content = ""
        for sentence in body.get_payload().split("\n"):
            if sentence.endswith('='):
                new_content += sentence[0:-1]
            else:
                new_content += sentence + "\n"

        """using regex to find links"""
        new_content = str.replace(new_content, '<', ' ')
        new_content = str.replace(new_content, '=2e', '.')
        new_content = str.replace(new_content, '=2E', '.')
        urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+\~]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', new_content)
        for url in urls:
            print(url)
            try:
                status = url_dict[url]
                print('caching', url+'...')
                return status
            except:
                if(re.match(r'.*(\.gif|\.jpg|\.jpeg|\.png)$', url)):
                    return False
                elif is_url_malicious(url):
                    return True
                time.sleep(1)
            return False



    else:
        return None
    return None

# Create mails

In [5]:
def create_mails(mails, name):
    if not os.path.exists('phish/'+name):
        os.mkdir('phish/'+name)
    for i, message in enumerate(mails):
        detected = False
        if(single_mail_malicious(message) == True):
            detected = True
        mail_file = open('phish/'+name+"/" + str(i) +"_"+str(detected)+".txt", "w")
        body = None
        print(i)
        tmp = []
        tmp.append('Subject: '+str(message['subject']))
        if message.is_multipart():
            for part in message.walk():
                if part.is_multipart():
                    for subpart in part.walk():
                        if subpart.get_content_type() == 'text/html':
                            body = subpart
                        elif subpart.get_content_type() == 'text/plain':
                            body = subpart
                        else:
                            continue
                elif part.get_content_type() == 'text/html':
                    body = part
                elif part.get_content_type() == 'text/plain':
                    body = part
                else:
                    continue


        elif message.get_content_type() == 'text/html':
            body = message
        elif message.get_content_type() == 'text/plain':
            body = message
        else:
            continue


        if (body == None or body.get_payload == None):
            mail_file.write("None" + '\n')
            none_file_set.add(i)
        elif body.get_content_type() == 'text/html':

            soup = BeautifulSoup(body.get_payload(),"html.parser")

            content = soup.get_text()
            content = content.encode('gbk', 'ignore')
            content = content.decode('gbk')
            content = content.replace('\n', ' ')
            content = content.replace('= ', '')

            for l in content.split():
                tmp.append(l.strip())
            tmp = ' '.join(tmp)
        elif body.get_content_type() == 'text/plain':
            tmp = body.get_payload()
        else:
            continue
        try:
            splited = split_sentence(tmp)
        except:
            enders = re.compile('[.!?]')
            sentencelist = enders.split(tmp)
            splited = '\n'.join(sentencelist)

        try:
            mail_file.write(splited)
        except:
            error_file_set.add(i)

In [6]:
none_file_set = set()
error_file_set = set()

In [7]:
file_list = ['20051114', 'phishing0', 'phishing1', 'phishing2', 'phishing3']

# Baseline

In [ ]:
#netcraft site report generation

print('Creating email database...')
for file in file_list:
    print(file)
    file_name = file
    mbox_file = file_name+'.mbox'
    mbox = mailbox.mbox(mbox_file)
    mails = [x for x in mbox]
    print('Email Count:', len(mails))
    create_mails(mails,file)

In [ ]:
ham_email_list = ['enron3','enron4', 'enron5', 'enron6']


In [ ]:
#enron netcraft check
def mail_malicious(folder, label):
    count = 0
    for f in os.listdir(os.path.join(folder,label)):
        count += 1
        with open(os.path.join(folder, label, f), 'r') as reader:
            try:
                c = reader.read()
            except:
                continue
            urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+\~]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', c)
            for url in urls:
                print(f)
                print(url)
                try:
                    status = url_dict[url]
                    print('caching', url+'...')
                    os.rename(folder+'/'+label+'/'+f, folder+'/'+label+'/'+f+'_'+status+'.txt')
                    return status
                except:
                    if(re.match(r'.*(\.gif|\.jpg|\.jpeg|\.png)$', url)):
                        return False
                    elif is_url_malicious(url):
                        os.rename(folder+'/'+label+'/'+f, folder+'/'+label+'/'+f+'_True.txt')
                        return True
                return False
    return None

In [ ]:
for folder in ham_email_list:
    mail_malicious(folder, 'ham')

In [ ]:
for folder in ham_email_list:
    mail_malicious(folder, 'spam')